# 이기적 빅데이터 분석기사 실기

## 기출문제 6회(2023.06.24 시행)

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score

* 학생 530명에 대한 학습용 데이터를 이용하여 학생구분 예측모형을 만든 후, 이를 평가용 데이터에 적용하여 얻은 230명 학생의 구분을 다음과 같은 형식의 CSV파일로 생성하시오.

In [2]:
# 구분예측
# 중학생
# 초등학생
# 대학생
# 고등학생
# ...

In [3]:
# 데이터 불러오기
train = pd.read_csv('C:/Users/User/Desktop/빅분기 연습/dataset-main/data_6_2_train.csv')
test = pd.read_csv('C:/Users/User/Desktop/빅분기 연습/dataset-main/data_6_2_test.csv')

display(train.head())
display(test.head())

,일평균수면시간,일평균학습시간,일평균스마트폰사용시간,키,몸무게,체격,기초대사량,거주지역,구분
0,7.35,7.74,4.8,Tall,52.050,L,1499.4,동구,고등학생
1,7.05,8.10,1.2,Tall,51.300,M,1624.4,서구,중학생
2,7.35,7.74,3.0,Tall,51.285,L,1499.4,동구,고등학생
3,8.58,6.66,4.8,Short,25.545,XS,1249.5,서구,초등학생
4,7.05,8.10,4.8,Tall,54.150,M,1624.4,서구,대학생


,일평균수면시간,일평균학습시간,일평균스마트폰사용시간,키,몸무게,체격,기초대사량,거주지역
0,7.97,7.11,1.2,Tall,51.375,L,1749.3,북구
1,9.80,5.76,1.2,Short,28.710,XS,1749.3,북구
2,7.66,7.38,1.2,Tall,36.915,L,1624.4,남구
3,9.50,5.94,3.0,Short,24.045,XS,1624.4,동구
4,6.44,8.82,3.0,Tall,45.180,S,1499.4,서구


### EDA

In [4]:
# 데이터정보 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      530 non-null    float64
 1   일평균학습시간      530 non-null    float64
 2   일평균스마트폰사용시간  530 non-null    float64
 3   키            530 non-null    object 
 4   몸무게          530 non-null    float64
 5   체격           530 non-null    object 
 6   기초대사량        530 non-null    float64
 7   거주지역         530 non-null    object 
 8   구분           530 non-null    object 
dtypes: float64(5), object(4)
memory usage: 37.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      230 non-null    float64
 1   일평균학습시간      230 non-null    float64
 2   일평균스마트폰사용시간  230 non-null    float64
 3   키            230 non-null    object 
 4   몸무게          230 non

In [5]:
# 학습용 데이터 기초통계 확인
train.describe(include = 'object')

,키,체격,거주지역,구분
count,530,530,530,530
unique,2,4,4,5
top,Tall,XS,남구,유치원생
freq,277,253,145,140


In [6]:
# 결측값 확인
train.isnull().sum()

일평균수면시간        0
일평균학습시간        0
일평균스마트폰사용시간    0
키              0
몸무게            0
체격             0
기초대사량          0
거주지역           0
구분             0
dtype: int64

In [7]:
# 타겟값 분포 확인
train['구분'].value_counts()

구분
유치원생    140
초등학생    121
고등학생    120
대학생      78
중학생      71
Name: count, dtype: int64

* 테스트 데이터에도 동일하게 수행

### 데이터 전처리

In [8]:
# 학습용데이터의 타겟값 처리
target = train.pop('구분')

In [9]:
# 학습용/테스트용 데이터의 원핫인코딩 진행
train_df = pd.get_dummies(train)
test_df = pd.get_dummies(test)
print(train_df.shape, test_df.shape)

(530, 15) (230, 15)


### 검증용데이터 생성

In [10]:
X_train, X_val, y_train, y_val = train_test_split(train_df, target, test_size = 0.2, random_state = 0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(424, 15) (106, 15) (424,) (106,)


### 분류모델 생성

* **predict**: 범주를 예측하여 반환 -> 회귀모델인 경우 0.3345...특정값 출력, 분류모델인 경우 0, 1, 2, 3 등 가장 확률이 높은 클래스 출력
* **predict_proba**: 분류모델이 각 클래스에 대한 확률 출력(회귀에는 사용 불가)

In [11]:
rf = RandomForestClassifier(random_state = 0)
rf.fit(X_train, y_train)
predict_val = rf.predict(X_val)
f1_score(y_val, predict_val, average = 'macro')

0.9317983948418732

### 테스트데이터로 예측, 결과 파일 생성

In [12]:
# 테스트용 데이터를 이용하여 예측
predict_test = rf.predict(test_df)
result = pd.DataFrame({'pred': predict_test})
print(result)

result.to_csv('result1.csv', index = False, encoding = 'utf-8-sig')

     pred
0    고등학생
1    초등학생
2     중학생
3    유치원생
4    고등학생
..    ...
225  초등학생
226  초등학생
227  고등학생
228  고등학생
229  초등학생

[230 rows x 1 columns]


## 기출문제 5회(2022.12.03 시행)

* used_car 데이터셋은 중고차와 관련된 정보를 가지고 있다. 주어진 학습용데이터를 이용하여 중고차의 판매 가격을 예측하고, 이를 평가용 데이터에 적용하여 얻은 예측결과를 다음과 같은 형식의 CSV 파일로 생성하시오.

In [13]:
# 데이터 불러오기
X_test = pd.read_csv('C:/Users/User/Desktop/빅분기 연습/dataset-main/used_car_X_test.csv')
X_train = pd.read_csv('C:/Users/User/Desktop/빅분기 연습/dataset-main/used_car_X_train.csv')
y_train = pd.read_csv('C:/Users/User/Desktop/빅분기 연습/dataset-main/used_car_y_train.csv')
display(X_train.head())

,id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,1,A1,2019,Automatic,3500,Petrol,145,40.9,2.0
1,2,RS4,2020,Semi-Auto,2500,Petrol,145,28.8,2.9
2,3,A8,2019,Semi-Auto,500,Diesel,145,40.4,3.0
3,4,Q5,2019,Semi-Auto,5089,Diesel,150,38.2,2.0
4,5,A5,2020,Semi-Auto,4951,Diesel,145,51.4,2.0


### EDA

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7468 non-null   int64  
 1   model         7468 non-null   object 
 2   year          7468 non-null   int64  
 3   transmission  7468 non-null   object 
 4   mileage       7468 non-null   int64  
 5   fuelType      7468 non-null   object 
 6   tax           7468 non-null   int64  
 7   mpg           7468 non-null   float64
 8   engineSize    7468 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 525.2+ KB


In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3200 non-null   int64  
 1   model         3200 non-null   object 
 2   year          3200 non-null   int64  
 3   transmission  3200 non-null   object 
 4   mileage       3200 non-null   int64  
 5   fuelType      3200 non-null   object 
 6   tax           3200 non-null   int64  
 7   mpg           3200 non-null   float64
 8   engineSize    3200 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 225.1+ KB


In [16]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      7468 non-null   int64
 1   price   7468 non-null   int64
dtypes: int64(2)
memory usage: 116.8 KB


In [17]:
# 결측값 확인
print(f'X_train : {X_train.isnull().sum().sum()}')
print(f'y_train : {y_train.isnull().sum().sum()}')
print(f'X_test : {X_test.isnull().sum().sum()}')

X_train : 0
y_train : 0
X_test : 0


In [18]:
# 데이터 나누기
# 범주형 데이터
X_train_word = X_train[['model', 'transmission', 'fuelType']]
X_test_word = X_test[['model', 'transmission', 'fuelType']]
print(X_train_word.head())

# 수치형 데이터
X_train_num = X_train.drop(columns = ['id', 'model', 'transmission', 'fuelType'])
X_test_num = X_test.drop(columns = ['id', 'model', 'transmission', 'fuelType'])
print(X_test_word.head())

  model transmission fuelType
0    A1    Automatic   Petrol
1   RS4    Semi-Auto   Petrol
2    A8    Semi-Auto   Diesel
3    Q5    Semi-Auto   Diesel
4    A5    Semi-Auto   Diesel
  model transmission fuelType
0    A4    Semi-Auto   Diesel
1    A6       Manual   Diesel
2    Q5    Semi-Auto   Diesel
3    A1       Manual   Petrol
4    A3       Manual   Petrol


### 데이터 전처리

In [19]:
# 데이터 스케일링
# MinMax 스케일러 생성
scaler = MinMaxScaler()

# 수치형 데이터에 MinMax 스케일링 적용
X_train_num_scale = scaler.fit_transform(X_train_num)
X_test_num_scale = scaler.fit_transform(X_test_num)

# 데이터프레임 설정
df_train_num = pd.DataFrame(X_train_num_scale, columns = X_train_num.columns)
df_test_num = pd.DataFrame(X_test_num_scale, columns = X_test_num.columns)

# 원핫 인코딩
df_train_word = pd.get_dummies(X_train_word)
df_test_word = pd.get_dummies(X_test_word)

In [20]:
# 훈련 데이터프레임 컬럼 목록
train_columns = set(df_train_word.columns)

# 테스트 데이터프레임 컬럼 목록
test_columns = set(df_test_word.columns)

# 훈련데이터에는 있지만 테스트 데이터에는 없는 컬럼 확인
missing_in_test = train_columns - test_columns

# 테스트데이터에는 있지만 훈련 데이터에는 없는 컬럼 확인
missing_in_train = test_columns - train_columns

print('컬럼 목록 중 테스트 데이터에 없는 컬럼: ', missing_in_test)
print('컬럼 목록 중 훈련 데이터에 없는 컬럼: ', missing_in_train)

컬럼 목록 중 테스트 데이터에 없는 컬럼:  {'model_ S5', 'model_ S8', 'model_ RS7', 'model_ A2'}
컬럼 목록 중 훈련 데이터에 없는 컬럼:  set()


In [21]:
# 컬럼 목록 중 없는 컬럼의 값을 0으로 해서 채워줌
df_test_word['model_ A2'] = 0
df_test_word['model_ S8'] = 0
df_test_word['model_ RS7'] = 0
df_test_word['model_ S5'] = 0

In [22]:
# 데이터 결합
df_train = pd.concat([df_train_num, df_train_word], axis = 1)
df_test = pd.concat([df_test_num, df_test_word], axis = 1)
print(df_train.head())

       year   mileage       tax       mpg  engineSize  model_ A1  model_ A2  \
0  0.954545  0.020825  0.250000  0.127811    0.317460       True      False   
1  1.000000  0.014874  0.250000  0.056213    0.460317      False      False   
2  0.954545  0.002970  0.250000  0.124852    0.476190      False      False   
3  0.954545  0.030283  0.258621  0.111834    0.317460      False      False   
4  1.000000  0.029461  0.250000  0.189941    0.317460      False      False   

   model_ A3  model_ A4  model_ A5  ...  model_ S8  model_ SQ5  model_ SQ7  \
0      False      False      False  ...      False       False       False   
1      False      False      False  ...      False       False       False   
2      False      False      False  ...      False       False       False   
3      False      False      False  ...      False       False       False   
4      False      False       True  ...      False       False       False   

   model_ TT  transmission_Automatic  transmission_Manua

### 검증용 데이터 생성

In [23]:
rf = RandomForestRegressor(random_state = 77)

X_train, X_val, y_train_t, y_val = train_test_split(df_train.values, y_train['price'].values, test_size = 0.3)

# 모델 학습
rf.fit(X_train, y_train_t)

# validation 데이터로 성능 평가
y_pred = rf.predict(X_val)
print(np.sqrt(mean_squared_error(y_val, y_pred)))

2535.463325101483


### 예측

In [24]:
y_pred = rf.predict(df_test)
df = pd.DataFrame(X_test['id'], columns = ['id'])
df['price'] = y_pred
print(df.head())

# CSV파일 생성
df.to_csv('result2.csv', index = False)

     id     price
0  7469  33664.06
1  7470  19018.59
2  7471  36489.61
3  7472  14979.18
4  7473  16551.26


C:\ProgramData\anaconda3\envs\pythonEx\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
